In [30]:
import pandas as pd
import numpy as np
import os 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, classification_report, precision_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import pickle

In [3]:
#afficher le répertoire courant
os.getcwd()
#Changer de répertoire de travail
os.chdir(r'C:\Users\CAPITAINE PIRLO\Downloads')

In [5]:
#Chargement du DF
data = pd.read_excel('Coeur.xlsx')
data.head()

AGE   SEXE  TDT  PAR  CHOLESTEROL  GAJ     ECG  FCMAX ANGINE  DEPRESSION   \
0   40  homme   AA  140          289    0  Normal    172    Non          0.0   
1   49  femme  DNA  160          180    0  Normal    156    Non          1.0   
2   37  homme   AA  130          283    0      ST     98    Non          0.0   
3   48  femme  ASY  138          214    0  Normal    108    Oui          1.5   
4   54  homme  DNA  150          195    0  Normal    122    Non          0.0   

       PENTE  CŒUR  
0  Ascendant     0  
1       Plat     1  
2  Ascendant     0  
3       Plat     1  
4  Ascendant     0

In [6]:
#Travailler avec une copie du data Frame pour éviter de le rechager en cas d'êrreur
df = data.copy()

In [7]:
#Afficher les informations relatives au DF
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AGE          918 non-null    int64  
 1   SEXE         918 non-null    object 
 2   TDT          918 non-null    object 
 3   PAR          918 non-null    int64  
 4   CHOLESTEROL  918 non-null    int64  
 5   GAJ          918 non-null    int64  
 6   ECG          918 non-null    object 
 7   FCMAX        918 non-null    int64  
 8   ANGINE       918 non-null    object 
 9   DEPRESSION   918 non-null    float64
 10  PENTE        918 non-null    object 
 11  CŒUR         918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [8]:
#Normalisation des variables quantitatives
for col in df.drop('CŒUR', axis =1).select_dtypes(np.number).columns:
    df[col] = df[col]/df[col].max()


In [9]:
#Vérification des résultats
df.head()

AGE   SEXE  TDT   PAR  CHOLESTEROL  GAJ     ECG     FCMAX ANGINE  \
0  0.519481  homme   AA  0.70     0.479270  0.0  Normal  0.851485    Non   
1  0.636364  femme  DNA  0.80     0.298507  0.0  Normal  0.772277    Non   
2  0.480519  homme   AA  0.65     0.469320  0.0      ST  0.485149    Non   
3  0.623377  femme  ASY  0.69     0.354892  0.0  Normal  0.534653    Oui   
4  0.701299  homme  DNA  0.75     0.323383  0.0  Normal  0.603960    Non   

   DEPRESSION       PENTE  CŒUR  
0     0.000000  Ascendant     0  
1     0.161290       Plat     1  
2     0.000000  Ascendant     0  
3     0.241935       Plat     1  
4     0.000000  Ascendant     0

In [10]:
#Encodage des variables qualitatives
for col in df.select_dtypes('object').columns:
    df[col] = df[col].astype('category').cat.codes

In [11]:
#Vérification des résultats
df.head()

AGE  SEXE  TDT   PAR  CHOLESTEROL  GAJ  ECG     FCMAX  ANGINE  \
0  0.519481     1    0  0.70     0.479270  0.0    1  0.851485       0   
1  0.636364     0    3  0.80     0.298507  0.0    1  0.772277       0   
2  0.480519     1    0  0.65     0.469320  0.0    2  0.485149       0   
3  0.623377     0    1  0.69     0.354892  0.0    1  0.534653       1   
4  0.701299     1    3  0.75     0.323383  0.0    1  0.603960       0   

   DEPRESSION   PENTE  CŒUR  
0     0.000000      0     0  
1     0.161290      2     1  
2     0.000000      0     0  
3     0.241935      2     1  
4     0.000000      0     0

In [12]:
#Séparer la variable cible (coeur) et les variables explicatives
y = df['CŒUR']
x = df.drop('CŒUR', axis = 1)

In [13]:
#Subdivision du jeu de données en apprentissage et en test
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =0.2, random_state =1)
#train_set, test_set = train_test_split(df,test_size =0.2, random_state =1)

In [14]:
#Vérifier le résultat
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(734, 11) (184, 11) (734,) (184,)


In [15]:
#Création d'un objet lr de la classe LogisticRegression
lr = LogisticRegression(solver ='newton-cg', random_state =1)

In [16]:
#Apprentissage du modèle
model = lr.fit(x_train, y_train)
#model = lr.fit(train_set.drop('CŒUR', axis = 1), train_set['CŒUR'])

In [17]:
#Probabilité d'appartenance à l'une des classes
predict_proba = model.predict_proba(x_test)
predict_proba[:5,:]

array([[0.39560595, 0.60439405],
       [0.18843348, 0.81156652],
       [0.04970935, 0.95029065],
       [0.09007617, 0.90992383],
       [0.28158313, 0.71841687]])

In [18]:
#Application du modèle au données de test
y_pred = model.predict(x_test)
y_pred[:5]

array([1, 1, 1, 1, 1], dtype=int64)

In [19]:
#matrice de confusion
mc = confusion_matrix(y_test, y_pred)
mc

array([[ 66,   8],
       [  6, 104]], dtype=int64)

In [20]:
#Taux de bonnes prédictions
accuracy = accuracy_score(y_test, y_pred)
accuracy
#Conclusion : le modèle produit 85.32 % de bonne prédiction.

0.9239130434782609

In [21]:
#Sensibilité
recall = recall_score(y_test, y_pred)
recall
#conclusion : lorsqu'un individu est malade, nous avons 84.11 % de chance que le modèle le prédise.

0.9454545454545454

In [22]:
#Précision
precision = precision_score(y_test, y_pred)
precision
#Conclusion : lorsque le modèle prédit qu'un individu est malade, nous avons 90 % de chace que cela soit vraie.

0.9285714285714286

In [23]:
#plus de détails avec classification report
cr = classification_report(y_test, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.92      0.89      0.90        74
           1       0.93      0.95      0.94       110

    accuracy                           0.92       184
   macro avg       0.92      0.92      0.92       184
weighted avg       0.92      0.92      0.92       184



In [24]:
#Modèle Trival
df['CŒUR'].value_counts()/df.shape[0]
#Conclusion : si nous nous basons sur le hasard pour faire nos prédiction, nous aurons 55.37 % de bonne prédiction.

1    0.553377
0    0.446623
Name: CŒUR, dtype: float64

In [25]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données d'entrainement)
model.score(x_train, y_train)

0.8433242506811989

In [26]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données de test)
model.score(x_test, y_test)

0.9239130434782609

In [27]:
# enregistrer le modèle
pickle.dump(lr, open('model.pkl', 'wb'))

arbre de décision


In [28]:
#copier notre dataframe dans une nouvelle variable
df2 = df.copy()
df2.head(10)

AGE  SEXE  TDT   PAR  CHOLESTEROL  GAJ  ECG     FCMAX  ANGINE  \
0  0.519481     1    0  0.70     0.479270  0.0    1  0.851485       0   
1  0.636364     0    3  0.80     0.298507  0.0    1  0.772277       0   
2  0.480519     1    0  0.65     0.469320  0.0    2  0.485149       0   
3  0.623377     0    1  0.69     0.354892  0.0    1  0.534653       1   
4  0.701299     1    3  0.75     0.323383  0.0    1  0.603960       0   
5  0.506494     1    3  0.60     0.562189  0.0    1  0.841584       0   
6  0.584416     0    0  0.65     0.393035  0.0    1  0.841584       0   
7  0.701299     1    0  0.55     0.344942  0.0    1  0.702970       0   
8  0.480519     1    1  0.70     0.343284  0.0    1  0.643564       1   
9  0.623377     0    0  0.60     0.470978  0.0    1  0.594059       0   

   DEPRESSION   PENTE  CŒUR  
0     0.000000      0     0  
1     0.161290      2     1  
2     0.000000      0     0  
3     0.241935      2     1  
4     0.000000      0     0  
5     0.000000      0     0  
6     0.000000      0     0  
7     0.000000      0     0  
8     0.241935      2     1  
9     0.000000      0     0

In [29]:
#Standardisation des variables quantitatives
for col in df2.drop('CŒUR', axis =1).select_dtypes(np.number).columns:
    df2[col] = df2[col]/df2[col].max()

In [31]:
#Encodage des variables qualitatives dans une variable le
le = LabelEncoder()

In [34]:
#creer une variable df_encoded qui va stocker notre dataframe encoder
df_encoded=df.iloc[:,0:12]

for i in df_encoded:
    df_encoded[i]=le.fit_transform(df_encoded[i])
    
df_encoded.head()

AGE  SEXE  TDT  PAR  CHOLESTEROL  GAJ  ECG  FCMAX  ANGINE  DEPRESSION   \
0   12     1    0   41          147    0    1     98       0           10   
1   21     0    3   55           40    0    1     82       0           20   
2    9     1    0   31          141    0    2     25       0           10   
3   20     0    1   39           72    0    1     34       1           25   
4   26     1    3   49           53    0    1     48       0           10   

   PENTE  CŒUR  
0      0     0  
1      2     1  
2      0     0  
3      2     1  
4      0     0

In [35]:
# jeu de fonctionnalités
x=df_encoded.iloc[:,0:11]
#l'étiquettement de notre jeu
y=df_encoded.iloc[:,11]


pour la separation de notre jeu de donnée on aurait pu procédé comme suit
#Séparer la variable cible (coeur) et les variables explicatives
y = df['CŒUR']
x = df.drop('CŒUR', axis = 1)

In [38]:
#repartition des données en données de test et données d'entraînement
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =0.2, random_state =1)

In [39]:
#Vérification de la séparation de notre jeu de donnée
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(734, 11) (184, 11) (734,) (184,)


In [40]:
#Création d'un objet model de la classe DecisionTreeClassifier
model_DT=DecisionTreeClassifier(criterion='gini')

In [41]:
#Apprentissage du modèle
model_DT.fit(x_train,y_train)

DecisionTreeClassifier()

In [45]:
#Probabilité d'appartenance à l'une des classes
predict_proba = model_DT.predict_proba(x_test)
predict_proba[:3,:]

array([[0., 1.],
       [1., 0.],
       [0., 1.]])

In [46]:
#Application du modèle au données de test
y_pred = model_DT.predict(x_test)
y_pred[:10]

array([1, 0, 1, 1, 0, 0, 0, 0, 0, 1], dtype=int64)

In [48]:
#matrice de confusion pour notre modèle d'arbre
mc = confusion_matrix(y_test, y_pred)
mc

array([[62, 12],
       [29, 81]], dtype=int64)

In [49]:
#Taux de bonnes prédictions de notre modèle
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.7771739130434783

In [50]:
#Sensibilité 
recall_DT = recall_score(y_test, y_pred)
recall_DT

0.7363636363636363

In [51]:
#Précision
precision_DT = precision_score(y_test, y_pred)
precision_DT

0.8709677419354839

In [52]:
#plus de détails avec classification report
cr_DT = classification_report(y_test, y_pred)
print(cr_DT)

              precision    recall  f1-score   support

           0       0.68      0.84      0.75        74
           1       0.87      0.74      0.80       110

    accuracy                           0.78       184
   macro avg       0.78      0.79      0.77       184
weighted avg       0.79      0.78      0.78       184



In [54]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données de test)
model_DT.score(x_test, y_test)

0.7771739130434783